In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import mean_absolute_error

In [2]:
df=pd.read_csv('data1.csv', index_col=[0])

In [3]:
data=df[['Id_rater', 'Id_beer', 'Rating']]

In [4]:
#take number of beer and number of rater to input in model
n_raters, n_beers = len(data.Id_rater.unique()), len(data.Id_beer.unique())
n_latent_factors = 3

In [5]:
def create_network(rater, beer, rating):
    
    #input beer and rate
    #embedding both beer and rater
    beer_input=Input(shape=[1],name='Beer')
    beer_embedding=Embedding(n_beers + 1, n_latent_factors, name='Movie-Embedding')(beer_input)
    beer_vec = Flatten(name='FlattenBeer')(beer_embedding)
    #add a drop-off layer to prevent overfitting
    beer_vec = keras.layers.Dropout(0.2)(beer_vec)
    
    rater_input=Input(shape=(1,),name='Rater')
    rater_embedding=Embedding(n_raters + 1, n_latent_factors, name='Rater-Embedding')(rater_input)
    rater_vec=Flatten(name='FlattenRater')(rater_embedding)
    rater_vec = keras.layers.Dropout(0.2)(rater_vec)
    
    #concat beer vec and rater vector to put in neural_network
    prod = Dot(name="Dot-Product", axes=1)([beer_vec, rater_vec])
    prod = keras.layers.Dropout(0.2)(prod)
    
    #3 hiddden laysers
    dense = keras.layers.Dense(100,name='FullyConnected')(prod)
    dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
    dense_2 = keras.layers.Dense(50,name='FullyConnected-1')(prod)
    dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
    dense_3 = keras.layers.Dense(25,name='FullyConnected-2', activation='relu')(dense_2)
    
    #output layer
    result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_3)
    
    adam = Adam(lr=0.005)
    model = keras.Model([rater_input, beer_input], result)
    model.compile(optimizer=adam,loss= 'mean_absolute_error')
    
    model.fit(x=[rater, beer], y=rating, batch_size=20, epochs=20, verbose=0)
    
    return model

In [6]:
train, test = train_test_split(data, test_size=0.2, random_state=10)

In [7]:
#run model
model=create_network(train.Id_rater, train.Id_beer, train.Rating)

In [8]:
#predict on test set
y_hat=model.predict([test.Id_rater, test.Id_beer])

In [9]:
y_true=test.Rating

In [10]:
#MAE was 0.46 means the prediction +- 0.46
print(mean_absolute_error(y_true, y_hat))

0.46765343997642134


In [11]:
#try predict model with random number in n_raters range
model.predict([pd.Series([123, 123]), pd.Series([345, 76])])

array([[3.4975214],
       [2.804327 ]], dtype=float32)

In [12]:
def recommendation(users_id): #define a function to predict 3 top beers for a user base on neural network model
    for user in users_id:
        df_rater=df[df['Id_rater']==int('{}'.format(user))].sort_values(by='Rating', ascending=False)
        #take the rater name
        rater=df_rater['Rater'].values[0]
        #take 3 beers that rater rated sort by their rating
        top3beer=df_rater['Beers'].values[:3]
        print("    Name of rater: {} ".format(rater))
        print('    - Drank beers: ')
        for beer in top3beer:
            print(beer)
        print('\n')
        print("   - Top 3 recommend beers : ")
        #make a rater series to equal beer series length
        #predict function take a pair of rater and beer to predict
        user_pre = pd.Series([int('{}'.format(user)) for i in range(df.Id_beer.nunique())])
        beer_pre = pd.Series(data.Id_beer.unique())
        predictions=model.predict([user_pre, beer_pre])
        #make a list of all rating
        predictions = np.array([a[0] for a in predictions])
        #take top 3 prediction beers
        recommended_beer_ids = (-predictions).argsort()[:3]
        for id_beer in recommended_beer_ids:
            df_beer=df[df['Id_beer']==int('{}'.format(id_beer))]
            beer_rec=df_beer['Beers'].values[0]
            print(beer_rec)
        print('\n')
        print('\n')

In [13]:
recommendation([12,34])

    Name of rater: ClinchMtnMan 
    - Drank beers: 
Traditional Lager
Oktoberfest


   - Top 3 recommend beers : 
Peacemaker Pilsner
Aecht Schlenkerla Rauchbier Märzen
Samuel Adams Cinder Bock




    Name of rater: sjrider 
    - Drank beers: 
Aecht Schlenkerla Rauchbier Urbock
1516 Kellerbier
Lager


   - Top 3 recommend beers : 
East Side Attitude
Hudepohl Oktoberfest Bier
Windhoek Light






In [15]:
recommendation([300,578,4683])

    Name of rater: ferrari29 
    - Drank beers: 
Speakerswagon Pilsner
Wisconsin Amber


   - Top 3 recommend beers : 
Kalnapilis 7.30
Melbourne Bitter
Bürger Light




    Name of rater: Jocose47 
    - Drank beers: 
Budweiser 1933 Repeal Reserve Amber Lager


   - Top 3 recommend beers : 
Peacemaker Pilsner
Aecht Schlenkerla Rauchbier Märzen
Super Awesome Lager




    Name of rater: kajerm 
    - Drank beers: 
Tuborg Classic
Svyturys Ekstra Draught


   - Top 3 recommend beers : 
Dark Chocolate Lager
Wiedemann’s Special Lager
East Side Attitude




